# Générateur d'attestation de déplacement dérogatoire

Voici un petit projet perso pour appliquer mes nouvelles connaissances sur la librairie Selenium utile pour le webscrapping dynamique, l'objectif est de générer une attestation de déplacement dérogatoire pour sortir de chez soi dans un rayon d'1km. En utilisant les coordonnées GPS de l'utilisateur, le programme va chercher sur Internet l'adresse correspondante à l'aide du site "coordonnees-gps.fr". Il va ensuite rentrer les informations de l'utilisateur (à coder) pour générer une attestation automatiquement sur le site du gouvernement "media.interieur.gouv.fr/deplacement-covid-19"

### Coordonnées utilisateur

In [1]:
import time
prenom = "Paul"
nom = "JOUET"
birthday = "02/03/2000"
birthplace = "Suresnes"
curtime = str(time.localtime()[3]) + ':' + str(time.localtime()[4])

La partie ci-dessus est la seule à modifier pour l'adapter à vos informations personnelles. Vous pouvez aussi décider de garder une adresse constante en modifiant manuellement les variables `adresse` `ville` et `zipcode`

### Importation des librairies Selenium

Selenium ainsi que le driver chrome doivent être installés au préalable dans l'environnement de travail 
- `!pip install selenium`
- [Chrome driver download](https://chromedriver.chromium.org/downloads)

In [2]:
from selenium import webdriver

In [3]:
from selenium.webdriver.support.ui import WebDriverWait

In [4]:
from selenium.webdriver.support import expected_conditions as ec

In [5]:
from selenium.webdriver.common.by import By

In [6]:
from selenium.webdriver.common.keys import Keys

### Lancement du driver

In [8]:
driver = webdriver.Chrome()

In [9]:
driver.execute_script("window.navigator.geolocation.getCurrentPosition = function(success){var position = {'coords' : {'latitude': '555','longitude': '999'}};success(position);}");

Ce script permet d'éviter un souci de blocage de la géolocalisation par le navigateur.
Trouvé [ici](https://stackoverflow.com/questions/8411816/how-do-i-enable-geolocation-support-in-chromedriver)

In [10]:
driver.get("https://www.coordonnees-gps.fr/")

In [11]:
time.sleep(2) # les time.sleep servent à s'assurer que la page a bien chargé avant de faire quoi que ce soit, pas toujours indispensable

#### Récupération de l'adresse selon la géolocalisation

In [12]:
waiter = WebDriverWait(driver, timeout=10)

driver.execute_script("window.scrollTo(0, 1000);")
# pour que l'adresse soit affichée sur le site, il faut scroller ce qui active la géolocalisation

ad = waiter.until(ec.presence_of_element_located(
    (By.CSS_SELECTOR,"h1[id='iwtitle']")
))
# le waiter permet de s'assurer que l'adresse est bien apparue à l'écran avant de récupérer son texte

txt=ad.text
txt2=txt.split(', ')

adresse = txt2[-3]
txt3 = txt2[-2].split(' ')
zipcode = txt3[0]
ville = txt3[1]
(adresse,zipcode,ville)

('Rue des Suisses', '92000', 'Nanterre')

Sur le site des coordonnées GPS, l'adresse est toujours formatée de la façon suivante : (elementfacultatif), (numRue) rue de Exemple, ZIPCODE City, France

#### Création de l'attestation

In [13]:
driver.get("https://media.interieur.gouv.fr/deplacement-covid-19/")

In [14]:
time.sleep(2)

In [15]:
waiter.until(ec.presence_of_element_located(
    (By.XPATH,"//button[contains(.,'Mettre')]")
)).click()

En arrivant sur le site un popup de mise à jour apparaît, on récupère le bouton via Xpath et on l'effectue en cliquant dessus

#### Remplissage des critères

On remplit simplement chacune des textbox avec les informations récupérées au préalable

In [16]:
prenom_field = driver.find_element_by_css_selector(
    "input[name='firstname']")
prenom_field.send_keys(prenom)

In [17]:
nom_field = driver.find_element_by_css_selector(
    "input[name='lastname']")
nom_field.send_keys(nom)

In [18]:
birthday_field = driver.find_element_by_css_selector(
    "input[name='birthday']")
birthday_field.send_keys(birthday)

In [19]:
birthplace_field = driver.find_element_by_css_selector(
    "input[name='placeofbirth']")
birthplace_field.send_keys(birthplace)

In [20]:
adresse_field = driver.find_element_by_css_selector(
    "input[name='address']")
adresse_field.send_keys(adresse)

In [21]:
ville_field = driver.find_element_by_css_selector(
    "input[name='city']")
ville_field.send_keys(ville)

In [22]:
zipcode_field = driver.find_element_by_css_selector(
    "input[name='zipcode']")
zipcode_field.send_keys(zipcode)

In [23]:
time_field = driver.find_element_by_css_selector("input[name='heuresortie']")
time_field.send_keys(curtime)

In [24]:
checkbox = driver.find_element_by_css_selector(
    "input[id='checkbox-sport_animaux']")
checkbox.click()

### Génération

L'attestation est générée par le site du gouvernement et téléchargée. Puis le driver est fermé.

In [25]:
generate_attestation = driver.find_element_by_css_selector("button[id='generate-btn']")
generate_attestation.click()

In [26]:
time.sleep(2)
driver.close()

Reste à faire : donner accès à ce programme via un site pour obtenir l'attestation sur une page internet mobile